In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v3 import CNN_v3

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v3 = CNN_v3((480, 640, 4), split_into_parts=20)
cnn_v3.def_model()

In [5]:
images = cnn_v3._prepare_images(image_train)
anwsers = cnn_v3._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v3.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
parallel_model.optimizer.lr.assign(1e-3)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.001>

In [14]:
path = "CNN/CNN_v3/weights/cnn_v3_1e_6_{}"

class ModelSaver(keras.callbacks.Callback):
    __path: str
    __current_epoch: int
        
    def __init__(self, path: str):
        self.__path = path
        self.__current_epoch = 0
        
    def on_epoch_end(self, *args, **kwargs):
        epoch_path = self.__path.format(self.__current_epoch)
        self.__current_epoch += 1
        
        parallel_model.save_weights(epoch_path)

model_saver = ModelSaver(path)

In [15]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=2,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/2
1244460/1244460 [==============================] - 116s 93us/step - loss: 20.0355 - accuracy: 0.5622 - val_loss: 2.3408 - val_accuracy: 0.2543
Epoch 2/2
1244460/1244460 [==============================] - 106s 85us/step - loss: 3.1029 - accuracy: 0.6827 - val_loss: 1.2809 - val_accuracy: 0.6742


In [16]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=3,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/3
1244460/1244460 [==============================] - 106s 86us/step - loss: 1.3130 - accuracy: 0.7133 - val_loss: 1.1933 - val_accuracy: 0.2519
Epoch 2/3
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.8651 - accuracy: 0.7316 - val_loss: 1.0199 - val_accuracy: 0.3026
Epoch 3/3
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.7201 - accuracy: 0.7460 - val_loss: 0.6624 - val_accuracy: 0.6971


In [17]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [18]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/10
1244460/1244460 [==============================] - 105s 84us/step - loss: 0.6814 - accuracy: 0.7531 - val_loss: 0.7615 - val_accuracy: 0.6541
Epoch 2/10
1244460/1244460 [==============================] - 103s 83us/step - loss: 0.6623 - accuracy: 0.7591 - val_loss: 0.8405 - val_accuracy: 0.6107
Epoch 3/10
1244460/1244460 [==============================] - 103s 83us/step - loss: 0.6504 - accuracy: 0.7634 - val_loss: 0.6997 - val_accuracy: 0.6773
Epoch 4/10
1244460/1244460 [==============================] - 103s 83us/step - loss: 0.6402 - accuracy: 0.7658 - val_loss: 0.7053 - val_accuracy: 0.6738
Epoch 5/10
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.6322 - accuracy: 0.7670 - val_loss: 0.6717 - val_accuracy: 0.6849
Epoch 6/10
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.6247 - accuracy: 0.7693 - val_loss: 0.6777 - val_accuracy: 0.6828
Epoch 7/10
1244460/1244460 [==

In [20]:
parallel_model.optimizer.lr.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [21]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5952 - accuracy: 0.7775 - val_loss: 0.6681 - val_accuracy: 0.6866
Epoch 2/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5933 - accuracy: 0.7781 - val_loss: 0.6571 - val_accuracy: 0.6922
Epoch 3/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5924 - accuracy: 0.7782 - val_loss: 0.6598 - val_accuracy: 0.6908
Epoch 4/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5915 - accuracy: 0.7786 - val_loss: 0.6621 - val_accuracy: 0.6896
Epoch 5/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5907 - accuracy: 0.7786 - val_loss: 0.6523 - val_accuracy: 0.6929


In [22]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [23]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5879 - accuracy: 0.7786 - val_loss: 0.6492 - val_accuracy: 0.6901
Epoch 2/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5837 - accuracy: 0.7798 - val_loss: 0.6392 - val_accuracy: 0.6945
Epoch 3/5
1244460/1244460 [==============================] - 104s 84us/step - loss: 0.5783 - accuracy: 0.7813 - val_loss: 0.6813 - val_accuracy: 0.6798
Epoch 4/5
1244460/1244460 [==============================] - 103s 83us/step - loss: 0.5721 - accuracy: 0.7833 - val_loss: 0.6679 - val_accuracy: 0.6890
Epoch 5/5
1244460/1244460 [==============================] - 103s 83us/step - loss: 0.5671 - accuracy: 0.7844 - val_loss: 0.6291 - val_accuracy: 0.6994


In [24]:
parallel_model.optimizer.lr.assign(1e-6)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-06>

In [25]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5641 - accuracy: 0.7853 - val_loss: 0.6286 - val_accuracy: 0.7010
Epoch 2/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5637 - accuracy: 0.7855 - val_loss: 0.6345 - val_accuracy: 0.6980
Epoch 3/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5637 - accuracy: 0.7852 - val_loss: 0.6321 - val_accuracy: 0.6987
Epoch 4/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5636 - accuracy: 0.7855 - val_loss: 0.6321 - val_accuracy: 0.6986
Epoch 5/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5633 - accuracy: 0.7855 - val_loss: 0.6320 - val_accuracy: 0.6984


In [26]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5630 - accuracy: 0.7856 - val_loss: 0.6313 - val_accuracy: 0.6991
Epoch 2/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5631 - accuracy: 0.7856 - val_loss: 0.6325 - val_accuracy: 0.6984
Epoch 3/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5632 - accuracy: 0.7856 - val_loss: 0.6286 - val_accuracy: 0.6999
Epoch 4/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5628 - accuracy: 0.7857 - val_loss: 0.6292 - val_accuracy: 0.6999
Epoch 5/5
1244460/1244460 [==============================] - 105s 84us/step - loss: 0.5627 - accuracy: 0.7857 - val_loss: 0.6287 - val_accuracy: 0.6995


In [27]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5627 - accuracy: 0.7858 - val_loss: 0.6279 - val_accuracy: 0.7000
Epoch 2/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5624 - accuracy: 0.7859 - val_loss: 0.6284 - val_accuracy: 0.6998
Epoch 3/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5622 - accuracy: 0.7858 - val_loss: 0.6304 - val_accuracy: 0.6989
Epoch 4/5
1244460/1244460 [==============================] - 105s 84us/step - loss: 0.5621 - accuracy: 0.7860 - val_loss: 0.6257 - val_accuracy: 0.7009
Epoch 5/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5619 - accuracy: 0.7861 - val_loss: 0.6261 - val_accuracy: 0.7007


In [28]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5620 - accuracy: 0.7864 - val_loss: 0.6282 - val_accuracy: 0.7001
Epoch 2/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5615 - accuracy: 0.7864 - val_loss: 0.6258 - val_accuracy: 0.7011
Epoch 3/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5617 - accuracy: 0.7864 - val_loss: 0.6298 - val_accuracy: 0.6995
Epoch 4/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5619 - accuracy: 0.7862 - val_loss: 0.6290 - val_accuracy: 0.7004
Epoch 5/5
1244460/1244460 [==============================] - 105s 84us/step - loss: 0.5619 - accuracy: 0.7859 - val_loss: 0.6287 - val_accuracy: 0.7006


In [29]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [30]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5605 - accuracy: 0.7860 - val_loss: 0.6420 - val_accuracy: 0.6919
Epoch 2/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5554 - accuracy: 0.7878 - val_loss: 0.6254 - val_accuracy: 0.7031
Epoch 3/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5726 - accuracy: 0.7742 - val_loss: 0.6653 - val_accuracy: 0.6615
Epoch 4/5
1244460/1244460 [==============================] - 105s 85us/step - loss: 0.5724 - accuracy: 0.7741 - val_loss: 0.5777 - val_accuracy: 0.7259
Epoch 5/5
1244460/1244460 [==============================] - 105s 84us/step - loss: 0.5500 - accuracy: 0.7869 - val_loss: 0.5613 - val_accuracy: 0.7317


In [31]:
parallel_model.optimizer.lr.assign(1e-6)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-06>

In [32]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=10000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5448 - accuracy: 0.7890 - val_loss: 0.6252 - val_accuracy: 0.7042
Epoch 2/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5450 - accuracy: 0.7890 - val_loss: 0.6283 - val_accuracy: 0.7029
Epoch 3/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5446 - accuracy: 0.7890 - val_loss: 0.6258 - val_accuracy: 0.7033
Epoch 4/5
1244460/1244460 [==============================] - 106s 85us/step - loss: 0.5447 - accuracy: 0.7889 - val_loss: 0.6196 - val_accuracy: 0.7070
Epoch 5/5
1244460/1244460 [==============================] - 106s 86us/step - loss: 0.5448 - accuracy: 0.7892 - val_loss: 0.6257 - val_accuracy: 0.7037
